In [1]:
"""
Airworthiness Directive Applicability System
Extracts and evaluates compliance rules from FAA and EASA ADs
"""

import json
from typing import List, Dict, Optional, Set
from dataclasses import dataclass, field, asdict
from enum import Enum

class ApplicabilityResult(Enum):
    APPLICABLE = "yes"
    NOT_APPLICABLE = "no"
    REQUIRES_INSPECTION = "requires_inspection"

@dataclass
class ModificationConstraint:
    """Represents a modification that affects applicability"""
    mod_number: Optional[str] = None
    service_bulletin: Optional[str] = None
    production_embodiment: bool = False  # If mod is embodied in production
    excludes_if_installed: bool = False  # If having this mod excludes aircraft
    requires_if_missing: bool = False  # If missing this mod makes it applicable

@dataclass
class MSNConstraint:
    """Represents MSN (Manufacturer Serial Number) constraints"""
    included_msns: List[str] = field(default_factory=list)
    excluded_msns: List[str] = field(default_factory=list)
    msn_ranges: List[Dict[str, int]] = field(default_factory=list)  # [{"min": x, "max": y}]

@dataclass
class ApplicabilityRules:
    """Complete applicability rules for an AD"""
    ad_id: str
    authority: str  # FAA or EASA
    issue_date: str
    effective_date: str
    aircraft_models: List[str]
    msn_constraints: Optional[MSNConstraint] = None
    modification_constraints: List[ModificationConstraint] = field(default_factory=list)
    additional_notes: List[str] = field(default_factory=list)

    def to_dict(self) -> Dict:
        """Convert to dictionary for JSON serialization"""
        result = {
            "ad_id": self.ad_id,
            "authority": self.authority,
            "issue_date": self.issue_date,
            "effective_date": self.effective_date,
            "aircraft_models": self.aircraft_models,
            "msn_constraints": None,
            "modification_constraints": [],
            "additional_notes": self.additional_notes
        }

        if self.msn_constraints:
            result["msn_constraints"] = {
                "included_msns": self.msn_constraints.included_msns,
                "excluded_msns": self.msn_constraints.excluded_msns,
                "msn_ranges": self.msn_constraints.msn_ranges
            }

        for mod in self.modification_constraints:
            result["modification_constraints"].append(asdict(mod))

        return result

@dataclass
class AircraftConfiguration:
    """Represents an aircraft configuration"""
    model: str
    msn: str
    modifications: List[str] = field(default_factory=list)

    def has_modification(self, mod_identifier: str) -> bool:
        """Check if aircraft has a specific modification"""
        # Check exact match or substring match
        for mod in self.modifications:
            if mod_identifier.lower() in mod.lower() or mod.lower() in mod_identifier.lower():
                return True
        return False

class ADApplicabilityEvaluator:
    """Evaluates aircraft configurations against AD rules"""

    def __init__(self):
        self.rules: Dict[str, ApplicabilityRules] = {}

    def load_rules(self, rules_data: List[Dict]):
        """Load AD rules from JSON data"""
        for rule_dict in rules_data:
            rule = self._dict_to_rules(rule_dict)
            self.rules[rule.ad_id] = rule

    def _dict_to_rules(self, data: Dict) -> ApplicabilityRules:
        """Convert dictionary to ApplicabilityRules"""
        msn_const = None
        if data.get("msn_constraints"):
            msn_data = data["msn_constraints"]
            msn_const = MSNConstraint(
                included_msns=msn_data.get("included_msns", []),
                excluded_msns=msn_data.get("excluded_msns", []),
                msn_ranges=msn_data.get("msn_ranges", [])
            )

        mod_constraints = []
        for mod_data in data.get("modification_constraints", []):
            mod_constraints.append(ModificationConstraint(**mod_data))

        return ApplicabilityRules(
            ad_id=data["ad_id"],
            authority=data["authority"],
            issue_date=data["issue_date"],
            effective_date=data["effective_date"],
            aircraft_models=data["aircraft_models"],
            msn_constraints=msn_const,
            modification_constraints=mod_constraints,
            additional_notes=data.get("additional_notes", [])
        )

    def evaluate(self, aircraft: AircraftConfiguration, ad_id: str) -> tuple[ApplicabilityResult, str]:
        """
        Evaluate if an aircraft is affected by an AD
        Returns: (result, explanation)
        """
        if ad_id not in self.rules:
            return ApplicabilityResult.NOT_APPLICABLE, f"AD {ad_id} not found in rules database"

        rule = self.rules[ad_id]

        # Step 1: Check aircraft model
        if not self._model_matches(aircraft.model, rule.aircraft_models):
            return ApplicabilityResult.NOT_APPLICABLE, f"Model {aircraft.model} not in affected models list"

        # Step 2: Check MSN constraints if present
        if rule.msn_constraints:
            msn_result, msn_reason = self._check_msn_constraints(aircraft.msn, rule.msn_constraints)
            if not msn_result:
                return ApplicabilityResult.NOT_APPLICABLE, msn_reason

        # Step 3: Check modification constraints
        if rule.modification_constraints:
            mod_result, mod_reason = self._check_modification_constraints(
                aircraft, rule.modification_constraints
            )
            if mod_result != ApplicabilityResult.APPLICABLE:
                return mod_result, mod_reason

        return ApplicabilityResult.APPLICABLE, "Aircraft matches all applicability criteria"

    def _model_matches(self, aircraft_model: str, affected_models: List[str]) -> bool:
        """Check if aircraft model matches any affected model"""
        aircraft_model_clean = aircraft_model.upper().replace("-", "").replace(" ", "")

        for affected in affected_models:
            affected_clean = affected.upper().replace("-", "").replace(" ", "")
            if aircraft_model_clean == affected_clean:
                return True
            # Check if it's a variant (e.g., A320-214 matches A320)
            if aircraft_model_clean.startswith(affected_clean):
                return True
            if affected_clean.startswith(aircraft_model_clean):
                return True

        return False

    def _check_msn_constraints(self, msn: str, constraints: MSNConstraint) -> tuple[bool, str]:
        """Check MSN constraints"""
        try:
            msn_int = int(msn)
        except ValueError:
            return False, f"Invalid MSN format: {msn}"

        # Check excluded MSNs
        if msn in constraints.excluded_msns or str(msn_int) in constraints.excluded_msns:
            return False, f"MSN {msn} is explicitly excluded"

        # Check included MSNs (if specified, only these are affected)
        if constraints.included_msns:
            if msn in constraints.included_msns or str(msn_int) in constraints.included_msns:
                return True, f"MSN {msn} is explicitly included"
            else:
                return False, f"MSN {msn} not in included MSN list"

        # Check MSN ranges
        if constraints.msn_ranges:
            in_range = False
            for range_spec in constraints.msn_ranges:
                if range_spec["min"] <= msn_int <= range_spec["max"]:
                    in_range = True
                    break
            if not in_range:
                return False, f"MSN {msn} not in affected ranges"

        return True, "MSN constraints satisfied"

    def _check_modification_constraints(
        self,
        aircraft: AircraftConfiguration,
        constraints: List[ModificationConstraint]
    ) -> tuple[ApplicabilityResult, str]:
        """Check modification constraints"""

        for constraint in constraints:
            mod_id = constraint.mod_number or constraint.service_bulletin

            if constraint.excludes_if_installed:
                # If aircraft has this mod, it's excluded
                if aircraft.has_modification(mod_id):
                    return (
                        ApplicabilityResult.NOT_APPLICABLE,
                        f"Aircraft has excluding modification: {mod_id}"
                    )

            if constraint.requires_if_missing:
                # If aircraft doesn't have this mod, it's affected
                if not aircraft.has_modification(mod_id):
                    return (
                        ApplicabilityResult.APPLICABLE,
                        f"Aircraft lacks required modification: {mod_id}"
                    )

            if constraint.production_embodiment:
                # This is informational - aircraft with this mod from production are excluded
                if aircraft.has_modification(mod_id):
                    return (
                        ApplicabilityResult.NOT_APPLICABLE,
                        f"Aircraft has production modification: {mod_id}"
                    )

        return ApplicabilityResult.APPLICABLE, "No excluding modifications found"


# EXTRACTED RULES DATA
AD_RULES_DATA = [
    {
        "ad_id": "FAA-2025-23-53",
        "authority": "FAA",
        "issue_date": "2025-11-14",
        "effective_date": "2025-12-01",
        "aircraft_models": [
            "MD-11",
            "MD-11F",
            "MD-10-10F",
            "MD-10-30F",
            "DC-10-10",
            "DC-10-10F",
            "DC-10-15",
            "DC-10-30",
            "DC-10-30F",
            "KC-10A",
            "KDC-10",
            "DC-10-40",
            "DC-10-40F"
        ],
        "msn_constraints": None,
        "modification_constraints": [],
        "additional_notes": [
            "Emergency AD due to engine pylon detachment accident",
            "Prohibits further flight until inspection and corrective actions completed",
            "Applies to all aircraft of these models regardless of MSN",
            "Supersedes Emergency AD 2025-23-51"
        ]
    },
    {
        "ad_id": "EASA-2025-0254",
        "authority": "EASA",
        "issue_date": "2025-11-28",
        "effective_date": "2025-12-08",
        "aircraft_models": [
            "A320-211",
            "A320-212",
            "A320-214",
            "A320-215",
            "A320-216",
            "A320-231",
            "A320-232",
            "A320-233",
            "A321-111",
            "A321-112",
            "A321-131"
        ],
        "msn_constraints": None,
        "modification_constraints": [
            {
                "mod_number": "24591",
                "service_bulletin": None,
                "production_embodiment": True,
                "excludes_if_installed": True,
                "requires_if_missing": False
            },
            {
                "mod_number": "24977",
                "service_bulletin": None,
                "production_embodiment": True,
                "excludes_if_installed": True,
                "requires_if_missing": False
            },
            {
                "mod_number": None,
                "service_bulletin": "A320-57-1089",
                "production_embodiment": False,
                "excludes_if_installed": True,
                "requires_if_missing": False
            },
            {
                "mod_number": None,
                "service_bulletin": "A320-57-1060",
                "production_embodiment": False,
                "excludes_if_installed": True,
                "requires_if_missing": False
            },
            {
                "mod_number": None,
                "service_bulletin": "A320-57-1088",
                "production_embodiment": False,
                "excludes_if_installed": True,
                "requires_if_missing": False
            },
            {
                "mod_number": None,
                "service_bulletin": "A320-57-1101",
                "production_embodiment": False,
                "excludes_if_installed": True,
                "requires_if_missing": False
            },
            {
                "mod_number": None,
                "service_bulletin": "A320-57-1126",
                "production_embodiment": False,
                "excludes_if_installed": True,
                "requires_if_missing": False
            },
            {
                "mod_number": None,
                "service_bulletin": "A320-57-1256",
                "production_embodiment": False,
                "excludes_if_installed": True,
                "requires_if_missing": False
            }
        ],
        "additional_notes": [
            "Concerns Wing Main Landing Gear Retraction Actuator Fitting inspection",
            "Supersedes EASA AD 2007-0162 and EASA AD 2014-0169",
            "Aircraft with qualifying modifications or SBs already applied are not affected",
            "Correction published to fix effective date"
        ]
    }
]


def main():
    """Main test function"""
    # Initialize evaluator
    evaluator = ADApplicabilityEvaluator()
    evaluator.load_rules(AD_RULES_DATA)

    # Test aircraft configurations
    test_aircraft = [
        AircraftConfiguration("MD-11", "48123", []),
        AircraftConfiguration("DC-10-30F", "47890", []),
        AircraftConfiguration("Boeing 737-800", "30123", []),
        AircraftConfiguration("A320-214", "5234", []),
        AircraftConfiguration("A320-232", "6789", ["mod 24591 (production)"]),
        AircraftConfiguration("A320-214", "7456", ["SB A320-57-1089 Rev 04"]),
        AircraftConfiguration("A321-111", "8123", []),
        AircraftConfiguration("A321-112", "364", ["mod 24977 (production)"]),
        AircraftConfiguration("A319-100", "9234", []),
        AircraftConfiguration("MD-10-10F", "46234", [])
    ]

    print("=" * 80)
    print("AIRWORTHINESS DIRECTIVE APPLICABILITY EVALUATION RESULTS")
    print("=" * 80)
    print()

    # Evaluate each aircraft against both ADs
    for aircraft in test_aircraft:
        print(f"Aircraft: {aircraft.model} (MSN: {aircraft.msn})")
        if aircraft.modifications:
            print(f"  Modifications: {', '.join(aircraft.modifications)}")
        else:
            print(f"  Modifications: None")
        print()

        # Evaluate against FAA AD
        faa_result, faa_reason = evaluator.evaluate(aircraft, "FAA-2025-23-53")
        print(f"  FAA AD 2025-23-53: {faa_result.value.upper()}")
        print(f"    Reason: {faa_reason}")

        # Evaluate against EASA AD
        easa_result, easa_reason = evaluator.evaluate(aircraft, "EASA-2025-0254")
        print(f"  EASA AD 2025-0254: {easa_result.value.upper()}")
        print(f"    Reason: {easa_reason}")

        print()
        print("-" * 80)
        print()

    # Export rules to JSON
    print("\n" + "=" * 80)
    print("RULES EXPORTED TO JSON FORMAT")
    print("=" * 80)
    print()

    rules_export = {
        "airworthiness_directives": [
            evaluator.rules[ad_id].to_dict()
            for ad_id in evaluator.rules
        ]
    }

    print(json.dumps(rules_export, indent=2))


if __name__ == "__main__":
    main()

AIRWORTHINESS DIRECTIVE APPLICABILITY EVALUATION RESULTS

Aircraft: MD-11 (MSN: 48123)
  Modifications: None

  FAA AD 2025-23-53: YES
    Reason: Aircraft matches all applicability criteria
  EASA AD 2025-0254: NO
    Reason: Model MD-11 not in affected models list

--------------------------------------------------------------------------------

Aircraft: DC-10-30F (MSN: 47890)
  Modifications: None

  FAA AD 2025-23-53: YES
    Reason: Aircraft matches all applicability criteria
  EASA AD 2025-0254: NO
    Reason: Model DC-10-30F not in affected models list

--------------------------------------------------------------------------------

Aircraft: Boeing 737-800 (MSN: 30123)
  Modifications: None

  FAA AD 2025-23-53: NO
    Reason: Model Boeing 737-800 not in affected models list
  EASA AD 2025-0254: NO
    Reason: Model Boeing 737-800 not in affected models list

--------------------------------------------------------------------------------

Aircraft: A320-214 (MSN: 5234)
  Mod